# 🌐 CORS verstehen und in Flask lösen

**Ziel:**  
Du lernst, was *CORS* ist, warum der Browser Anfragen blockiert und wie du das Problem im eigenen Flask-Projekt behebst.

**Themen:**
1. Was ist CORS?
2. Wann tritt das Problem auf?
3. Wie erkennt man es?
4. Wie löst man es mit Flask?
5. Praktische Beispiele


## 1️⃣ Was ist CORS?

**CORS** steht für **Cross-Origin Resource Sharing**.  
Es ist ein **Sicherheitsmechanismus im Browser**, der verhindert, dass eine Webseite einfach Daten von einem anderen Server lädt.

### 🧩 Beispiel
Wenn du eine HTML-Seite lokal öffnest (z. B. mit Live Server auf Port 5500):

```
http://127.0.0.1:5500
```

und dein Flask-Server läuft auf:

```
http://127.0.0.1:5000
```

dann stammen sie **von unterschiedlichen Origins**  
(die *Origin* besteht aus **Protokoll + Host + Port**).

➡️ Der Browser blockiert solche Anfragen automatisch – es erscheint ein Fehler wie:

```
Access to fetch at 'http://127.0.0.1:5000/api/test' 
from origin 'http://127.0.0.1:5500' has been blocked by CORS policy.
```


## 💻 Warum wir `http.server` brauchen

Wenn du eine HTML-Datei einfach **per Doppelklick** öffnest, läuft sie im Browser mit der Adresse:

```
file:///C:/Users/Schueler/Desktop/index.html
```

Das ist **keine echte Webseite**, sondern eine Datei aus dem Dateisystem.  
Der Browser sendet dann den Header:

```
Origin: null
```

Das führt bei einer Anfrage an Flask zu einem **CORS-Fehler**, weil Flask nur Anfragen über HTTP erlaubt.

---

### ✅ Lösung: Mini-Webserver starten

Mit Python kannst du ganz einfach einen kleinen Server starten, der die Dateien korrekt über **HTTP** ausliefert:

```bash
cd pfad/zum/ordner
python -m http.server 5500
```

Danach erreichst du deine Seite über:

```
http://127.0.0.1:5500
```

Jetzt sendet der Browser den Header:

```
Origin: http://127.0.0.1:5500
```

Flask kann diese Origin erlauben, und **CORS funktioniert** endlich korrekt.

---

### 🧠 Merke:
> `http.server` sorgt dafür, dass deine Seite wie eine echte Webanwendung läuft –  
> der Browser behandelt sie dann „normal“ und blockiert die Anfragen nicht mehr.


## 2️⃣ Wann tritt das Problem auf?

Das Problem tritt **immer dann** auf, wenn:
- dein **Frontend** (HTML, JS) von einer anderen Adresse läuft als dein **Backend (Flask)**, oder
- du deine HTML-Datei direkt per `file://` öffnest (dann ist die *Origin* „null“).

### Typische Fälle
| Frontend | Backend | Ergebnis |
|-----------|----------|-----------|
| `http://127.0.0.1:5500` | `http://127.0.0.1:5000` | ❌ CORS-Fehler |
| `file:///C:/index.html` | `http://127.0.0.1:5000` | ❌ CORS-Fehler („Origin null“) |
| `http://127.0.0.1:5000` (beides Flask) | `http://127.0.0.1:5000` | ✅ Kein Problem |


## 3️⃣ Wie löst man das Problem in Flask?

Wir verwenden das Paket **flask-cors**, das automatisch die richtigen HTTP-Header hinzufügt.

### 🔧 Schritt 1: Installation
```bash
pip install flask-cors
```

### 🔧 Schritt 2: Anwendung im Code
Die einfachste Variante erlaubt alle Domains:
```python
from flask import Flask, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # erlaubt alle Origins

@app.get("/api/test")
def test():
    return jsonify(message="Hallo von Flask!")

app.run(debug=True)
```

Damit funktioniert auch eine Anfrage von einer anderen Portnummer oder `file://`.

💡 Diese Variante ist für **den Unterricht und lokale Tests** völlig in Ordnung.


## 4️⃣ Sichere Variante (nur bestimmte Origins erlauben)

Für reale Projekte solltest du die erlaubten *Origins* einschränken:

```python
from flask import Flask, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app, origins=[
    "http://127.0.0.1:5500",
    "http://localhost:5500"
])

@app.get("/api/test")
def test():
    return jsonify(ok=True, msg="Nur erlaubte Origins dürfen zugreifen.")

app.run(port=5000, debug=True)
```

So dürfen nur Anfragen von diesen Adressen auf die API zugreifen.


## 5️⃣ Praktische Übung

1. Starte deinen Flask-Server mit dem obigen Code.
2. Öffne deine HTML/JS-Seite mit **VS Code → Live Server** (Port 5500).
3. Führe im Browser‑Konsolenfenster aus:

```js
fetch("http://127.0.0.1:5000/api/test")
  .then(r => r.json())
  .then(console.log)
  .catch(console.error);
```

Wenn alles richtig ist, erscheint in der Konsole:
```
{ ok: true, msg: "Nur erlaubte Origins dürfen zugreifen." }
```

---

🧠 **Merksatz:**
> Wenn der Browser eine Anfrage blockiert, liegt es fast immer an CORS –  
> die Lösung ist, dem Server explizit mitzuteilen, welche *Origins* vertrauenswürdig sind.


## ✅ Zusammenfassung

| Begriff | Bedeutung |
|----------|------------|
| **Origin** | Kombination aus Protokoll, Host und Port |
| **CORS** | Mechanismus, der den Zugriff von anderen Origins blockiert |
| **flask-cors** | Bibliothek, die passende Header automatisch hinzufügt |
| **Lösung** | `CORS(app)` oder `CORS(app, origins=[...])` |

---

### ⚡ Bonus:
Wenn du dein HTML direkt aus Flask auslieferst (z. B. mit `send_from_directory`),  
kommt alles von derselben Origin → **kein CORS nötig**.


---
---
---
# Vorgehen

1. HTML-Datei erstellen und Code schreiben
2. JavaScript-Datei ersellen und Code schreiben
3. Webseite öffnen  
(Der Server (und die client-Datei) sollten auf die Eingaben auf der Webseite reagieren)

---
---

## Was man beachten muss, wenn es nicht funktioniert

---

### 1. flask-cors installieren
Man muss ins Terminal `pip install flask-cors` schreiben

---

### 2. Origins erlauben
Mann muss in der Server-Datei alle oder nur bestimmte Origins erlauben, damit eine Anfrage auch von anderen Portnummern oder von `file://` aus geht:

2.1
```python
from flask_cors import CORS
```
2.2
```python
# Variante 1
app = Flask(__name__)
CORS(app)  #erlaubt alle Origins

# Variante 2 (FUNKTIONIERT NICHT!)
CORS(app, origins=[
    "http://127.0.0.1:5500",    #erlaubt nur diese Origins
    "http://localhost:5500"
])
```

---

### 3. Neue Route erstellen
Man muss/ kann in der Server-Datei eine neue Route z.B. Formular erstellen. Sie verknüpft die Webseite mit dem Server, so dass man über den Server die Datei Webseite öffnen kann. Wenn diese Route aufgerufen wird, zeigt der Browser die Webseite an.

3.1
```python
from flask import send_from_directory
```
3.2
```python
@app.route('/formular')
def formular():
    return send_from_directory('../08_04_Webanwendung', 'a_webseite.html')
```

---

### 4. Public
Unter PORTS müssen der Server/ alle Ports auf Public gesetzt werden

---

### 5. Webseite aufrufen
Schritt 1: Server starten
Schritt 2: Ordner des Java Script Clients im Terminal aufrufen (cd pfad/zum/ordner)
Schritt 3: 'python -m http.server 5500' ins Terminal schreiben
Schritt 4: Auf den Button rechts unten 'Im Browser öffnen' klicken
Schritt 5: Alle Ports auf public setzen